In [1]:
import requests
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Read in and Aggregate County Data

In [2]:
ia12 = pd.read_excel("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/Iowa_2012_cds.xlsx")

In [3]:
ia12 = ia12["County District Population"].str.split(' ', expand=True)

In [4]:
ia12.columns = ["county","district_112","pop"]

In [5]:
ia12 = ia12[["county","district_112"]]

In [6]:
ia12.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/iowa/iowa_2012.csv", index = False)

After manually imputing county names via the published map (thanks IA leg), I have a csv of the states districts

In [7]:
ia_cts = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/Iowa_2022.csv")

In [8]:
#remove the underscore used to prevent deliniation
ia_cts["county"] = ia_cts["county"].str.replace("_"," ")

In [9]:
fips = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/iowa_fips.csv")
fips["county"] = fips["county"].replace("O Brien","O'Brien")

In [10]:
ia_cts = pd.merge(ia_cts,fips, on="county", how="outer")

In [11]:
ia_cts["ct_FIPS"] = ia_cts["FIPS"].astype(str).str[2:]

# Read in and Aggregate District Data

In [12]:
d = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/aggregated_fairness.csv")
#s = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/2022_redraw_workbook.csv")

In [13]:
d = d[d["ST"] == "IA"]
d = d[d["year"] > 2018]
#s = s[s["ST"] == "IA"]

In [14]:
data_118 = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/full_districts/data_118.csv")
data_117 = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/full_districts/data_117.csv")

In [15]:
data_118 = data_118[data_118["ST"] == "IA"]
data_117 = data_117[data_117["ST"] == "IA"]
dist = pd.concat([data_118,data_117])
dist["district"] = dist["ST#"].str[-1]
dist

,year,congress,ST,ST#,PVI,metric,seat,is_GOP,district
103,2022,118,IA,IA1,R+2,0.52,NaN,NaN,1
104,2022,118,IA,IA2,R+3,0.53,NaN,NaN,2
105,2022,118,IA,IA3,R+1,0.51,NaN,NaN,3
106,2022,118,IA,IA4,R+14,0.64,NaN,NaN,4
159,2020,117,IA,IA1,R+4,0.54,R,1.0,1
160,2020,117,IA,IA2,R+4,0.54,R,1.0,2
161,2020,117,IA,IA3,R+3,0.53,D,0.0,3
162,2020,117,IA,IA4,R+16,0.66,R,1.0,4


In [16]:
dicts = pd.read_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/metric_converter.csv")
dists = pd.merge(dist,dicts, on="metric")

In [17]:
dists = dists.sort_values("year").sort_values("district").reset_index(drop=True)
dists["Chance of GOP Representation"] = (dists["prob_GOP"]*100).round()
dists["key"] = dists["district"].astype(str) + "-" + dists["year"].astype(str)

In [37]:
#setup mean fields to automatically adjust
pg_22 = dists[dists["year"]==2022].prob_GOP.mean()
pg_20 = dists[dists["year"]==2020].prob_GOP.mean()
m_22 = dists[dists["year"]==2022].metric.mean()
m_20 = dists[dists["year"]==2020].metric.mean()
pvi_22 = "R+" + str(round((m_22-.5)*100))
pvi_20 = "R+" + str(round((m_20-.5)*100))

'R+5'

In [32]:
sums = pd.DataFrame({'year' : [2020, 2022],
           'ST#' : ["Mean", "Mean"],
           "PVI" : [pvi_20, pvi_22],
            "metric" : [m_20, m_22],
           "Chance of GOP Representation" : [pg_20, pg_22]
            })
print_dists = pd.concat([dists,sums])
print_dists

,year,congress,ST,ST#,PVI,metric,seat,is_GOP,district,pvi_range,prob_GOP,Chance of GOP Representation,key
0,2020,117.0,IA,IA1,R+4,0.5400,R,1.0,1,R+4,0.788991,79.000000,1-2020
1,2022,118.0,IA,IA1,R+2,0.5200,NaN,NaN,1,R+2,0.653846,65.000000,1-2022
2,2020,117.0,IA,IA2,R+4,0.5400,R,1.0,2,R+4,0.788991,79.000000,2-2020
3,2022,118.0,IA,IA2,R+3,0.5300,NaN,NaN,2,R+3,0.688172,69.000000,2-2022
4,2020,117.0,IA,IA3,R+3,0.5300,D,0.0,3,R+3,0.688172,69.000000,3-2020
5,2022,118.0,IA,IA3,R+1,0.5100,NaN,NaN,3,R+1,0.591837,59.000000,3-2022
6,2020,117.0,IA,IA4,R+16,0.6600,R,1.0,4,R+8 to R+20,0.953509,95.000000,4-2020
7,2022,118.0,IA,IA4,R+14,0.6400,NaN,NaN,4,R+8 to R+20,0.953509,95.000000,4-2022
0,2020,NaN,NaN,Mean,R+6.75,0.5675,NaN,NaN,NaN,NaN,NaN,0.804916,NaN
1,2022,NaN,NaN,Mean,R+5.000000000000004,0.5500,NaN,NaN,NaN,NaN,NaN,0.721841,NaN


In [20]:
print_dists.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/iowa_districts.csv", index = False)

In [21]:
iact_112 = ia_cts[["county","district_112","FIPS","ct_FIPS"]]
iact_112["year"] = 2020
iact_112["congress"] = 117
iact_112 = iact_112.rename(columns={"district_112":"district"})
iact_118 = ia_cts[["county","district_118","FIPS","ct_FIPS"]]
iact_118["year"] = 2022
iact_118["congress"] = 118
iact_118 = iact_118.rename(columns={"district_118":"district"})

<ipython-input-21-b9346a0515cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iact_112["year"] = 2020
<ipython-input-21-b9346a0515cc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iact_118["year"] = 2022


In [22]:
iact = pd.concat([iact_112,iact_118])
iact["key"] = iact["district"].astype(str) + "-" + iact["year"].astype(str)
iact = iact.drop(columns=["district","year","congress"])
df = pd.merge(iact,dists,on="key")

In [23]:
df = df.drop(columns=["key","seat","is_GOP"])
df.head()

,county,FIPS,ct_FIPS,year,congress,ST,ST#,PVI,metric,district,pvi_range,prob_GOP,Chance of GOP Representation
0,Allamakee,19005,005,2020,117,IA,IA1,R+4,0.54,1,R+4,0.788991,79.0
1,Benton,19011,011,2020,117,IA,IA1,R+4,0.54,1,R+4,0.788991,79.0
2,Black Hawk,19013,013,2020,117,IA,IA1,R+4,0.54,1,R+4,0.788991,79.0
3,Bremer,19017,017,2020,117,IA,IA1,R+4,0.54,1,R+4,0.788991,79.0
4,Buchanan,19019,019,2020,117,IA,IA1,R+4,0.54,1,R+4,0.788991,79.0


In [24]:
df_118 = df[df["congress"]==118]
df_117 = df[df["congress"]==117][["county","district","PVI","prob_GOP"]]
df_117.columns = ["county","district_117","PVI_117","prob_GOP_117"]
df_118 = pd.merge(df_118, df_117, on="county")
df_118

,county,FIPS,ct_FIPS,year,congress,ST,ST#,PVI,metric,district,pvi_range,prob_GOP,Chance of GOP Representation,district_117,PVI_117,prob_GOP_117
0,Allamakee,19005,005,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
1,Benton,19011,011,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
2,Black Hawk,19013,013,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
3,Bremer,19017,017,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
4,Buchanan,19019,019,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
5,Clayton,19043,043,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
6,Delaware,19055,055,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
7,Dubuque,19061,061,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
8,Fayette,19065,065,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991
9,Howard,19089,089,2022,118,IA,IA2,R+3,0.53,2,R+3,0.688172,69.0,1,R+4,0.788991


In [25]:
#df.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/iowa.csv", index = False)
df_118.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/iowa.csv", index = False)
df[df["congress"]==117].to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/iowa_117.csv", index = False)

In [26]:
df.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/iowa_slider.csv", index = False)